# Deep Network
## Rede neural com duas camadas ocultas para o problema de classificação de digitos utilizando o framework TensorFlow
## A base de dados é a MNIST com 10 classes (dígitos de 0 a 9)

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

In [3]:
tf.GraphKeys.VARIABLES = tf.GraphKeys.GLOBAL_VARIABLES

(1) Definir a Arquitetura da rede:
    - Camada de entrada = número de features = número de pixels da imagem
      As imagens da MNIST tem 28x28 pixels, ou seja 784 features
    - Camada de saída = número de classes = número de dígitos
      Na MNIST temos 10 dígitos, de 0 a 9, ou seja 10 neurônios de saída
     

In [4]:
# placeholder : alocacao de tamanho indefinido
# variable : alocacao de tamanho pre-definido

# tamanho das camadas ocultas
L1 = 256 # camada oculta 1
L2 = 128 # camada oculta 2
O  = 10  # camada de saida (output)

# matriz de entrada
# placeholder do tipo float, com numero indefinido de imagens porem todas de tamanho 28x28x1
X = tf.placeholder(tf.float32, [None, 28, 28, 1])

In [6]:
# Primeira camada oculta (inicializacao aleatoria usando distribuicao normal)
# matriz de pesos - 784 x L1 (784 features x L1 neuronios da camada oculta 1)
W1 = tf.Variable(tf.truncated_normal([784, L1], stddev=0.1))
# termos bias - L1, um para cada neuronio
b1 = tf.Variable(tf.truncated_normal([L1], stddev=0.1))

# Segunda camada oculta
# matriz de pesos - L1 x L2 (256 x 128 neuronios)
W2 = tf.Variable(tf.truncated_normal([L1, L2], stddev=0.1))
# termos bias - L2, um para cada neuronio
b2 = tf.Variable(tf.truncated_normal([L2], stddev=0.1))

# Camada de saida (idem, agora transformando para as 10 classes finais)
W3 = tf.Variable(tf.truncated_normal([L2, 10], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([10], stddev=0.1))

In [11]:
# camada de saida (rotulos reais)
Y = tf.placeholder(tf.float32, [None, 10])

(2) Definir como serão feitos os cálculos;
uso de multiplicação matricial! 

$$f_1(\mathbf{x}_1) = \operatorname{relu}(W_1\mathbf{x}_1 + \mathbf{b}_1) = \mathbf{x}_2$$
$$f_2(\mathbf{x}_2) =  \operatorname{relu}(W_2\mathbf{x}_2 + \mathbf{b}_2) = \mathbf{x}_3$$
$$f_3(\mathbf{x}_3) = \operatorname{softmax}(W_3\mathbf{x}_3 + \mathbf{b}_3) = \hat{\mathbf{y}}$$

inicializar os pesos e outras variaveis

In [24]:
init = tf.global_variables_initializer() # instancia inicializacao

In [13]:
# As predicoes sao dadas pela formula anterior
X1 = tf.reshape(X, [-1, 784])  # vetoriza as imagens

X2 = tf.nn.relu(tf.matmul(X1,W1) + b1)
X3 = tf.nn.relu(tf.matmul(X2,W2) + b2)

# formula para as probabilidades de saida
X4 = tf.matmul(X3,W3) + b3
Y_ = tf.nn.softmax( X4 )

(3) Funcao de custo: informa quão longe estamos da solução ideal
    Entropia cruzada
    $$\sum_{i=0}^{N} -(Y \cdot \log(\hat Y))$$

In [28]:
crossEnt = tf.nn.softmax_cross_entropy_with_logits(logits=X4, labels=Y)
crossEnt = tf.reduce_mean(crossEnt)*batchSize

In [15]:
# alem da funcao de custo vamos calcular tambem a acuracia
# Y[i]  = [0   , 0  , 0  , 1  ,  0  , 0  , 0  , 0  , 0  , 0  ] -> vetor rotulo real
# X4[i] = [0.1 , 0  , 0.2, 0.7,  0  , 0  , 0  , 0  , 0  , 0  ] -> probabilidades preditas
# Y_[i] = [0   , 0  , 0 ,  1  ,  0  , 0  , 0  , 0  , 0  , 0  ] -> rotulo predito pela rede
correctPred = tf.equal( tf.argmax(Y_,1), tf.argmax(Y,1) )
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

(4) Metodo de otimizacao e suas variaveis

    *Gradiente Descendente* ou versões: RMSProp, Adam, etc.
    *Taxa de Aprendizado* = 0.003
    *Batch size* = 64

In [21]:
lrate = 0.003
batchSize = 64
iterations = 5

optMethod = tf.train.AdamOptimizer(lrate) # metodo alternativo de otimizacao
trainProcess = optMethod.minimize(crossEnt)

(5) Rodar um experimento com a MNIST

In [22]:
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [30]:
# Sessao TensorFlow
sess = tf.Session() # instancia a sessao
sess.run(init) # inicializa variaveis com o objeto que criamos anteriormente (init)

In [46]:
for i in range(iterations):
    # carrega batch um par Imagem, Rotulo
    batX, batY = mnist.train.next_batch(batchSize)
    # passar para o tensorflow, preciso definir um dicionario
    # o dicionario contem o batch no formato Chave, Valor: Imagens, Rotulos
    # a 'chave' deve ser a mesma que definimos como placeholders para a arquitetura
    # nesse caso X e Y
    trainData = {X: batX, Y: batY}
    # executa uma iteracao (feed-forward e backpropagation)
    sess.run(trainProcess, feed_dict=trainData)
    
    loss = sess.run(crossEnt, feed_dict=trainData)
    acc  = sess.run(accuracy, feed_dict=trainData)
    print(str(i) + " Loss: " + str(loss) + " Accuracy: "+str(acc*100)+"%")

0 Loss: 33.9248 Accuracy: 89.0625%
1 Loss: 18.2534 Accuracy: 89.0625%
2 Loss: 19.3333 Accuracy: 90.625%
3 Loss: 21.0635 Accuracy: 90.625%
4 Loss: 24.354 Accuracy: 95.3125%


(6) Avaliar o modelo em dados nao vistos no treinamento

In [47]:
testData = {X: mnist.test.images, Y: mnist.test.labels}
accTest  = sess.run(accuracy, feed_dict=testData)
print("Accuracy on test data: " + str(accTest*100)+"%")

Accuracy on test data: 91.1099970341%
